In [ ]:
import os
import glob
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime
from geospatial_plotting import plot_region, REGION_BOUNDS

import sys;       sys.path.append('../util/shared/python/')

from helper import read_ObsFcstAna, read_tilecoord, get_tile_species_obs_values

In [ ]:
expt_name_1 = 'DAv8_M36_Aus'

start_date = datetime(2021, 6, 1)
end_date = datetime(2021, 6, 30)

start_date_str = start_date.strftime('%Y/%m/%d')
end_date_str = end_date.strftime('%Y/%m/%d')


ana_directory_1 = f'/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/CYGNSS_Experiments/DAv8_M36_Aus_v3/DAv8_M36_Aus/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2021/M06'


In [ ]:
species_number = 12

OFA_list = []

OFA_list.extend(read_ObsFcstAna(fname) for fname in sorted(glob.glob(os.path.join(ana_directory_1, 'DAv8_M36_Aus.ens_avg.ldas_ObsFcstAna.20210611_1500z.bin'))))

# Initialize lists to store filtered data
all_tilenums = []
all_species = []
all_lats = []
all_lons = []
all_obs = []
all_fcst = []
all_ana = []

# Process each element in OFA_list
for ofa in OFA_list:
    # Create mask for this element
    assim_mask = ofa['obs_assim'] == 1 
    species_mask = ofa['obs_species'] == species_number 
    combined_mask = assim_mask & species_mask
    
    # Append filtered data
    all_tilenums.append(ofa['obs_tilenum'][combined_mask])
    all_species.append(ofa['obs_species'][combined_mask])
    all_lats.append(ofa['obs_lat'][combined_mask])
    all_lons.append(ofa['obs_lon'][combined_mask])
    all_obs.append(ofa['obs_obs'][combined_mask])
    all_fcst.append(ofa['obs_fcst'][combined_mask])
    all_ana.append(ofa['obs_ana'][combined_mask])


# Create filtered dictionary with concatenated data
filtered_data = {
    'obs_tilenum': np.concatenate(all_tilenums),
    'obs_species': np.concatenate(all_species),
    'obs_lat': np.concatenate(all_lats),
    'obs_lon': np.concatenate(all_lons),
    'obs_obs': np.concatenate(all_obs),
    'obs_fcst': np.concatenate(all_fcst),
    'obs_ana': np.concatenate(all_ana)
}



In [ ]:
tc = read_tilecoord('/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/CYGNSS_Experiments/DAv8_M36_Aus/DAv8_M36_Aus/output/SMAP_EASEv2_M36_GLOBAL/rc_out/DAv8_M36_Aus.ldas_tilecoord.bin')
n_tile = tc['N_tile']
lat = tc['com_lat']
lon = tc['com_lon']

map_array = np.empty([n_tile, 3])
map_array.fill(np.nan)
map_array[:, 1] = lon
map_array[:, 2] = lat


In [ ]:
cat_dir = '/Users/amfox/Desktop/GEOSldas_diagnostics/test_data/CYGNSS_Experiments/DAv8_M36_Aus_v3/DAv8_M36_Aus/output/SMAP_EASEv2_M36_GLOBAL/cat/ens_avg/Y2021/M06'

incr_file = os.path.join(cat_dir, 'DAv8_M36_Aus.inst3_1d_lndfcstana_Nt.20210611.nc4')

incr_ds = xr.open_dataset(incr_file)

# Extract SFMC_FCST and SFMC_ANA and time_stamp
sfmc_fcst = incr_ds['SFMC_FCST'].values
sfmc_ana = incr_ds['SFMC_ANA'].values
time_stamp = incr_ds['time_stamp'].values

In [ ]:
# Decode byte strings and convert to datetime objects
time_stamp_decoded = [datetime.strptime(ts.decode('utf-8'), '%Y%m%d_%H%Mz') for ts in time_stamp]

# Print the converted timestamps
print(time_stamp_decoded)

In [ ]:
obs_lon = filtered_data['obs_lon']
obs_lat = filtered_data['obs_lat']
obs_obs = filtered_data['obs_obs']
obs_fcst = filtered_data['obs_fcst']
obs_ana = filtered_data['obs_ana']

n_obs = obs_lon.shape[0]

In [ ]:

array = np.column_stack([sfmc_fcst[5, :], lon, lat])
#array = np.column_stack([np.full_like(sfmc_fcst[5, :], np.nan), lon, lat])
#array[np.abs(array[:, 0]) < 0.0001, 0] = np.nan

overlay = np.column_stack([obs_obs, obs_lon, obs_lat])

fig, ax = plot_region(
    array,
    region_bounds=REGION_BOUNDS['australia'],
    plot_title='Surface SM Forecast: 20210611_1500z',
   # overlay_points=overlay,
    units='m³/m³',
    cmin=-.0,
    cmax=.3,
)
fig.tight_layout()
plt.savefig('surface_sm_fcst_australia_20210611.png', dpi=300, bbox_inches='tight')

array = np.column_stack([np.full_like(sfmc_fcst[5, :], np.nan), lon, lat])

fig, ax = plot_region(
    array,
    region_bounds=REGION_BOUNDS['australia'],
    plot_title='Surface SM CYGNSS Obs: 20210611_1500z',
    overlay_points=overlay,
    units='m³/m³',
    cmin=-.0,
    cmax=.3,
)
fig.tight_layout()
plt.savefig('surface_sm_obs_australia_20210611.png', dpi=300, bbox_inches='tight')

array = np.column_stack([sfmc_fcst[5, :], lon, lat])

fig, ax = plot_region(
    array,
    region_bounds=REGION_BOUNDS['australia'],
    plot_title='Surface SM Obs & Forecast: 20210611_1500z',
    overlay_points=overlay,
    units='m³/m³',
    cmin=-.0,
    cmax=.3,
)
fig.tight_layout()
plt.savefig('surface_sm_obs_fcst_australia_20210611.png', dpi=300, bbox_inches='tight')

array = np.column_stack([np.full_like(sfmc_fcst[5, :], np.nan), lon, lat])
overlay = np.column_stack([obs_obs - obs_fcst, obs_lon, obs_lat])

fig, ax = plot_region(
    array,
    region_bounds=REGION_BOUNDS['australia'],
    plot_title='Surface SM OmF: 20210611_1500z',
    overlay_points=overlay,
    units='m³/m³',
    cmin=-.03,
    cmax=.03,
)
fig.tight_layout()
plt.savefig('surface_sm_omf_australia_20210611.png', dpi=300, bbox_inches='tight')

array = np.column_stack([sfmc_ana[5, :] - sfmc_fcst[5, :], lon, lat])
array[np.abs(array[:, 0]) < 0.0001, 0] = np.nan

fig, ax = plot_region(
    array,
    region_bounds=REGION_BOUNDS['australia'],
    plot_title='Surface SM OmF & Increment: 20210611_1500z',
    overlay_points=overlay,
    units='m³/m³',
    cmin=-.03,
    cmax=.03,
)
fig.tight_layout()
plt.savefig('surface_sm_ana_incr_australia_20210611.png', dpi=300, bbox_inches='tight')

fig, ax = plot_region(
    array,
    region_bounds=REGION_BOUNDS['australia'],
    plot_title='Surface SM Increment: 20210611_1500z',
    # overlay_points=overlay,
    units='m³/m³',
    cmin=-.03,
    cmax=.03,
)
fig.tight_layout()
plt.savefig('surface_sm_incr_australia_20210611.png', dpi=300, bbox_inches='tight')

array = np.column_stack([sfmc_ana[5, :], lon, lat])

fig, ax = plot_region(
    array,
    region_bounds=REGION_BOUNDS['australia'],
    plot_title='Surface SM Analysis: 20210611_1500z',
    # overlay_points=overlay,
    units='m³/m³',
    cmin=-.0,
    cmax=.3,
)
fig.tight_layout()
plt.savefig('surface_sm_ana_australia_20210611.png', dpi=300, bbox_inches='tight')